In [1]:
import pandas as pd

markets_info_df = pd.read_parquet("epl_markets.parquet")
all_histories = pd.read_parquet("token_histories.parquet")
markets_info_df.dtypes

winner                  string[python]
loser                   string[python]
is_draw                        boolean
question                string[python]
condition_id            string[python]
question_id             string[python]
id                      string[python]
description             string[python]
end_date_iso            string[python]
uma_end_data            string[python]
volume                         Float64
closed                         boolean
first_token_id          string[python]
first_token_outcome     string[python]
first_token_price              Float64
second_token_id         string[python]
second_token_outcome    string[python]
second_token_price             Float64
game_start_time         datetime64[ns]
dtype: object

In [12]:
markets_info_df

,winner,loser,is_draw,question,condition_id,question_id,id,description,end_date_iso,uma_end_data,volume,closed,first_token_id,first_token_outcome,first_token_price,second_token_id,second_token_outcome,second_token_price,game_start_time,match_id
0,Manchester United,Liverpool,False,Will Manchester United beat Liverpool?,0x0f69c362da4dddd255e8bb6496a45190a2617037066d...,0x79449fb8b681725667d7778f00955ef196cc51213dc1...,505841,This market refers to the EPL match between Ma...,2024-09-01,2024-09-01T20:11:24Z,27321.177281,True,1070268098614301483769400035048192945868028748...,Yes,0.0,4641946105864093247918510751580466890037570902...,No,1.0,2024-09-01 15:00:00,8823695792809006390
1,Crystal Palace,Chelsea,False,Will Crystal Palace beat Chelsea?,0x2bf1b0179d6a9a5860bbfe2f6e940a4bfe5098b40dd6...,0xcd5eb7b63d13f593a7ff8c5e660234395dab1cd61742...,505834,This market refers to the EPL match between Ch...,2024-09-01,2024-09-01T17:46:58Z,16465.558516,True,2240836493145815788231779528861370764772082802...,Yes,0.0,9112607163262702891896524070679080530791126243...,No,1.0,2024-09-01 12:30:00,2972098483726502574
2,Newcastle United,Tottenham Hotspur,True,Will Newcastle vs. Tottenham end in a draw?,0x932c6ffb2641359184ee1384a0e0f3b3263abef2c2c4...,0xb5123c439793bca6fcc5ba6a27c53149347f3c8c6c58...,505838,This market refers to the EPL match between Ne...,2024-09-01,2024-09-01T17:46:58Z,1648.157327,True,4968230879603371948866041796987077791752868535...,Yes,0.0,6486150380988866473748206016838559835962169397...,No,1.0,2024-09-01 12:30:00,4769646815217436346
3,Chelsea,Crystal Palace,False,Will Chelsea beat Crystal Palace?,0x1ce48df226891b8526f7ae474ff6222353e661029f0e...,0xcd5eb7b63d13f593a7ff8c5e660234395dab1cd61742...,505833,This market refers to the EPL match between Ch...,2024-09-01,2024-09-01T17:45:57Z,74848.61748,True,8473141659136127648919471372081023410636831516...,Yes,0.0,7168237386744928574044957256855095466455619768...,No,1.0,2024-09-01 12:30:00,2972098483726502574
4,Newcastle United,Tottenham Hotspur,False,Will Newcastle beat Tottenham?,0xae59919d3f9c80ec9a6275fea57f1b81122ca0146a81...,0xb5123c439793bca6fcc5ba6a27c53149347f3c8c6c58...,505836,This market refers to the EPL match between Ne...,2024-09-01,2024-09-01T17:51:08Z,16197.184165,True,3902396297125861646265048524779791779669292294...,Yes,1.0,6390143151867843714655104214159797394603648846...,No,0.0,2024-09-01 12:30:00,4769646815217436346
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
219,Arsenal,Bournemouth,False,Will Arsenal beat Bournemouth?,0xbcffa2bee01c74202918565b5c830e5d58a8ff84572b...,0xabee421fb7998bce94aa2e6a655fb78a4cd7e9550d3a...,252437,"In the upcoming EPL game, scheduled for Septem...",2023-09-30,"September 30, 2023",37.138885,True,3476227583566235893385207197809170546483891999...,Yes,1.0,6626472848259693355623768601741507876320467959...,No,0.0,2023-09-30 14:00:00,-2481250318176953905
220,Crystal Palace,Wolverhampton Wanderers,False,Will Crystal Palace beat Wolves?,0x0bf3663ff0086dff8a29e57e92ef6b69b92f685fb931...,0x6cfc72e99b0657ff1988954e458e380060588769fa7f...,252147,In the upcoming Premier League soccer match sc...,2023-09-02,2023-09-03 17:02:02+00,0.068964,True,3991272135719335455425689967467591073248100202...,Yes,1.0,4095702638071914562064741827563675366981703730...,No,0.0,2023-09-03 11:00:00,-2145596344615660699
221,Chelsea,Fulham,False,Will Chelsea beat Fulham?,0x1df6000c8d55e48c41bf110e36d0863822df91b7701d...,0xd01609a3367e5431623af303a5f28ce5c6c1e7fe93d4...,252443,"In the upcoming EPL game, scheduled for Octobe...",2023-10-02,"October 2, 2023",0.0,True,4332907715665797429263582313875876304976304556...,Yes,1.0,6814765515250501822276957562102569149744638626...,No,0.0,2023-10-02 19:00:00,-289364474366586025
222,Chelsea,Nottingham Forest,False,Will Chelsea beat Nottingham Forest?,0x98cd21fa93bd6222cd8ec6c91616b3402a1ea61b1ed3...,0xb516f4dd44d1609168aad69fb9ae49e58d2743feed77...,252143,In the upcoming Premier League soccer match sc...

In [2]:
markets_info_df = markets_info_df.assign(
    match_id=markets_info_df[["winner", "loser", "game_start_time"]]
    .agg(frozenset, axis=1)
    .apply(hash)
)

In [24]:
markets_info_df = markets_info_df.drop_duplicates().loc[markets_info_df.volume > 0.01]

In [42]:
for match_id in markets_info_df.match_id:
    match_df = markets_info_df.loc[markets_info_df.match_id == match_id]
    assert len(match_df) == 3

    draw_row = match_df[match_df.is_draw].iloc[0]
    first_winlose_row = match_df[~match_df.is_draw].iloc[0]
    second_winlose_row = match_df[~match_df.is_draw].iloc[1]

    output_row = (
        first_winlose_row.drop(
            [
                "is_draw",
                "question",
                "condition_id",
                "question_id",
                "id",
                "description",
                "end_date_iso",
                "uma_end_data",
                "first_token_outcome",
                "second_token_outcome",
            ]
        )
        .rename({"winner": "first", "loser": "second"})
        .rename(
            {
                "volume": "first_win_volume",
                "first_token_id": "first_win_yes_token_id",
                "first_token_price": "first_win_yes_token_price",
                "second_token_id": "first_win_no_token_id",
                "second_token_price": "first_win_no_token_price",
            }
        )
    )
    
    second_row_rename_dict = {
                "volume": "second_win_volume",
                "first_token_id": "second_win_yes_token_id",
                "first_token_price": "second_win_yes_token_price",
                "second_token_id": "second_win_no_token_id",
                "second_token_price": "second_win_no_token_price",
            }
    
    second_winlose_row = second_winlose_row.loc[list(second_row_rename_dict.values())]

    print(second_winlose_row)
    break

KeyError: "None of [Index(['second_win_volume', 'second_win_yes_token_id',\n       'second_win_yes_token_price', 'second_win_no_token_id',\n       'second_win_no_token_price'],\n      dtype='object')] are in the [index]"

In [3]:
traded_df = markets_info_df[markets_info_df.volume > 0.01].drop_duplicates()
markets_info_traded_df_draws = (
    traded_df[traded_df.is_draw]
    .drop(columns=[c for c in traded_df.columns if "outcome" in c])
    .rename(columns={c: c.replace("first", "draw_yes") for c in traded_df.columns})
    .rename(columns={c: c.replace("second", "draw_no") for c in traded_df.columns})
    .drop(
        columns=[
            "is_draw",
            "question",
            "condition_id",
            "question_id",
            "id",
            "description",
            "end_date_iso",
            "uma_end_data",
        ]
    )
)
markets_info_traded_df_draws

,winner,loser,volume,closed,draw_yes_token_id,draw_yes_token_price,draw_no_token_id,draw_no_token_price,game_start_time,match_id
2,Newcastle United,Tottenham Hotspur,1648.157327,True,4968230879603371948866041796987077791752868535...,0.0,6486150380988866473748206016838559835962169397...,1.0,2024-09-01 12:30:00,4769646815217436346
5,Chelsea,Crystal Palace,10562.379214,True,6628081043611276835629741132699692180128963103...,1.0,2038204275565301344807054547965085333069777021...,0.0,2024-09-01 12:30:00,2972098483726502574
8,Manchester City,West Ham United,4246.952856,True,4652444992635632642529930521199192482880908236...,0.0,5519563340748631936641826338054313097421920479...,1.0,2024-08-31 16:30:00,477538746085173143
11,Aston Villa,Leicester City,6130.092364,True,7198006125524017729020999112949591754130072873...,0.0,1108105314871755573169486973066833564111072150...,1.0,2024-08-31 14:00:00,-952614444799699451
13,Bournemouth,Everton,2343.687726,True,6979466265655015180736494637283075984297819167...,0.0,9201488080747150865847341565897594405713204637...,1.0,2024-08-31 14:00:00,-7758498606970748066
15,Fulham,Ipswich Town,1958.268762,True,1056239115982502641842514644390919491396338064...,1.0,6474528836620100582363221559257100699919142728...,0.0,2024-08-31 14:00:00,-8801890004257238758
17,Nottingham Forest,Wolverhampton Wanderers,2678.878368,True,1922636093149757117752916299704362461806023417...,1.0,1072383393901019078745413673212653811002433746...,0.0,2024-08-31 14:00:00,4099726658047049859
25,Arsenal,Brighton & Hove Albion,8872.87221,True,3282101053200081500732589041854931815637585813...,1.0,5646917549123102449163419527608818678170441410...,0.0,2024-08-31 11:30:00,-8762489910048187249
26,Brentford,Southampton,502.707692,True,3102412140979869086108516105922538862811635984...,0.0,1071771006010210786837770353476734639789497155...,1.0,2024-08-31 14:00:00,-6514747391177184247
30,Brentford,Liverpool,9199.510944,True,4937274880027886183807479622344736653709925151...,0.0,8933757800840528696678082589188567084286739295...,1.0,2024-08-25 15:30:00,-7146698704919939652


In [4]:
traded_wl_df = (
    traded_df[~traded_df.is_draw]
    .drop(columns=[c for c in traded_df.columns if "outcome" in c])
    .rename(columns={c: c.replace("first", "first_win_yes") for c in traded_df.columns})
    .rename(columns={c: c.replace("second", "first_win_no") for c in traded_df.columns})
    .drop(
        columns=[
            "is_draw",
            "question",
            "condition_id",
            "question_id",
            "id",
            "description",
            "end_date_iso",
            "uma_end_data",
        ]
    )
)

traded_wl_df_switched = (
    traded_df[~traded_df.is_draw]
    .drop(columns=[c for c in traded_df.columns if "outcome" in c])
    .rename(columns={"winner": "loser", "loser": "winner"})
    .rename(
        columns={c: c.replace("first", "second_win_yes") for c in traded_df.columns}
    )
    .rename(
        columns={c: c.replace("second", "second_win_no") for c in traded_df.columns}
    )
    .drop(
        columns=[
            "is_draw",
            "question",
            "condition_id",
            "question_id",
            "id",
            "description",
            "end_date_iso",
            "uma_end_data",
        ]
    )
)

traded_wl_df_switched

,loser,winner,volume,closed,second_win_yes_token_id,second_win_yes_token_price,second_win_no_token_id,second_win_no_token_price,game_start_time,match_id
0,Manchester United,Liverpool,27321.177281,True,1070268098614301483769400035048192945868028748...,0.0,4641946105864093247918510751580466890037570902...,1.0,2024-09-01 15:00:00,8823695792809006390
1,Crystal Palace,Chelsea,16465.558516,True,2240836493145815788231779528861370764772082802...,0.0,9112607163262702891896524070679080530791126243...,1.0,2024-09-01 12:30:00,2972098483726502574
3,Chelsea,Crystal Palace,74848.61748,True,8473141659136127648919471372081023410636831516...,0.0,7168237386744928574044957256855095466455619768...,1.0,2024-09-01 12:30:00,2972098483726502574
4,Newcastle United,Tottenham Hotspur,16197.184165,True,3902396297125861646265048524779791779669292294...,1.0,6390143151867843714655104214159797394603648846...,0.0,2024-09-01 12:30:00,4769646815217436346
6,Tottenham Hotspur,Newcastle United,5338.051969,True,5462649995618436636028509551748704905681350869...,0.0,5130458727082798673488813540528413902741629816...,1.0,2024-09-01 12:30:00,4769646815217436346
...,...,...,...,...,...,...,...,...,...,...
214,Liverpool,Aston Villa,62.783485,True,6039475446325558590610830792923439094235664256...,1.0,7478682910843262796119864286568811611031017045...,0.0,2023-09-03 11:00:00,548005272248197253
215,Arsenal,Manchester United,24.285713,True,5719121657815100684863998168411340463668945474...,1.0,4633691986224411951686891585684086012813294716...,0.0,2023-09-03 13:30:00,-5936764714993044095
219,Arsenal,Bournemouth,37.138885,True,3476227583566235893385207197809170546483891999...,1.0,6626472848259693355623768601741507876320467959...,0.0,2023-09-30 14:00:00,-2481250318176953905
220,Crystal Palace,Wolverhampton Wanderers,0.068964,True,3991272135719335455425689967467591073248100202...,1.0,4095702638071914562064741827563675366981703730...,0.0,2023-09-03 11:00:00,-2145596344615660699


In [5]:
traded_wl_df

,winner,loser,volume,closed,first_win_yes_token_id,first_win_yes_token_price,first_win_no_token_id,first_win_no_token_price,game_start_time,match_id
0,Manchester United,Liverpool,27321.177281,True,1070268098614301483769400035048192945868028748...,0.0,4641946105864093247918510751580466890037570902...,1.0,2024-09-01 15:00:00,8823695792809006390
1,Crystal Palace,Chelsea,16465.558516,True,2240836493145815788231779528861370764772082802...,0.0,9112607163262702891896524070679080530791126243...,1.0,2024-09-01 12:30:00,2972098483726502574
3,Chelsea,Crystal Palace,74848.61748,True,8473141659136127648919471372081023410636831516...,0.0,7168237386744928574044957256855095466455619768...,1.0,2024-09-01 12:30:00,2972098483726502574
4,Newcastle United,Tottenham Hotspur,16197.184165,True,3902396297125861646265048524779791779669292294...,1.0,6390143151867843714655104214159797394603648846...,0.0,2024-09-01 12:30:00,4769646815217436346
6,Tottenham Hotspur,Newcastle United,5338.051969,True,5462649995618436636028509551748704905681350869...,0.0,5130458727082798673488813540528413902741629816...,1.0,2024-09-01 12:30:00,4769646815217436346
...,...,...,...,...,...,...,...,...,...,...
214,Liverpool,Aston Villa,62.783485,True,6039475446325558590610830792923439094235664256...,1.0,7478682910843262796119864286568811611031017045...,0.0,2023-09-03 11:00:00,548005272248197253
215,Arsenal,Manchester United,24.285713,True,5719121657815100684863998168411340463668945474...,1.0,4633691986224411951686891585684086012813294716...,0.0,2023-09-03 13:30:00,-5936764714993044095
219,Arsenal,Bournemouth,37.138885,True,3476227583566235893385207197809170546483891999...,1.0,6626472848259693355623768601741507876320467959...,0.0,2023-09-30 14:00:00,-2481250318176953905
220,Crystal Palace,Wolverhampton Wanderers,0.068964,True,3991272135719335455425689967467591073248100202...,1.0,4095702638071914562064741827563675366981703730...,0.0,2023-09-03 11:00:00,-2145596344615660699


In [13]:
merged_duped = (
    traded_wl_df.merge(
        traded_wl_df_switched,
        on=["winner", "loser", "game_start_time", "match_id"],
        suffixes=("_first_win", "_second_win"),
    )
    .merge(
        markets_info_traded_df_draws,
        on=["winner", "loser", "game_start_time", "match_id"],
        suffixes=("", "_draw"),
    )
    .rename(columns={"winner": "first", "loser": "second"})
    .groupby("match_id")
    .first()
    .reset_index()
)
merged_duped

,match_id,first,second,volume_first_win,closed_first_win,first_win_yes_token_id,first_win_yes_token_price,first_win_no_token_id,first_win_no_token_price,game_start_time,...,second_win_yes_token_id,second_win_yes_token_price,second_win_no_token_id,second_win_no_token_price,volume,closed,draw_yes_token_id,draw_yes_token_price,draw_no_token_id,draw_no_token_price
0,-8801890004257238758,Fulham,Ipswich Town,5815.749542,True,5390684612591871180226940463614602007632608985...,0.0,4835867034043260323210602260980380172342860099...,1.0,2024-08-31 14:00:00,...,2724863763841420578046708779868589387441936666...,0.0,4592496941083629569447042476295961754389643201...,1.0,1958.268762,True,1056239115982502641842514644390919491396338064...,1.0,6474528836620100582363221559257100699919142728...,0.0
1,-8772681732634810279,Bournemouth,Nottingham Forest,1011.850958,True,1050823326941745808232230384776635914686556222...,0.0,3055705040419724043925898401765169215229404101...,1.0,2024-08-17 14:00:00,...,6136814017597483222485589924157953297800237271...,0.0,1429174952419877936659470081835158792207990521...,1.0,11762.437828,True,5548849022392522826256320455747757471022017206...,1.0,1287798470161381994661331765600389966380379915...,0.0
2,-8762489910048187249,Arsenal,Brighton & Hove Albion,38583.198239,True,6111503160491403443221654004462481775096015381...,0.0,5891194826219718921556265951457901437359904302...,1.0,2024-08-31 11:30:00,...,1116610378897274981341652849429823672930913311...,0.0,5259322497049373785466208713290228068678828929...,1.0,8872.87221,True,3282101053200081500732589041854931815637585813...,1.0,5646917549123102449163419527608818678170441410...,0.0
3,-8627603966281271525,Arsenal,Aston Villa,45331.94036,True,5407858762055642414194294314062033095094505113...,1.0,1926331838542292358683977471374101091601070425...,0.0,2024-08-24 16:30:00,...,3572690946983071319451542669524457204608893230...,0.0,9508566054309741253938951248227178834216390840...,1.0,5577.057493,True,3984949472265518240895263056129376916184709390...,0.0,4485337955163659016002637467309784252215764770...,1.0
4,-7758498606970748066,Bournemouth,Everton,4898.660583,True,3959980066742036215089748788937411215762900226...,1.0,2171311100446299993519861709116600368641588070...,0.0,2024-08-31 14:00:00,...,8414157994261804734944978451273314491500711434...,0.0,4058076631458650619417463379827423945891322141...,1.0,2343.687726,True,6979466265655015180736494637283075984297819167...,0.0,9201488080747150865847341565897594405713204637...,1.0
5,-7146698704919939652,Brentford,Liverpool,10460.211613,True,8058812937056295301077754963716989842392959855...,0.0,4586745120104545026109118389516543779698202352...,1.0,2024-08-25 15:30:00,...,6181134725108988943382990745734226687444082523...,1.0,2689521720270122027066650781604176657297852439...,0.0,9199.510944,True,4937274880027886183807479622344736653709925151...,0.0,8933757800840528696678082589188567084286739295...,1.0
6,-6514747391177184247,Brentford,Southampton,8893.260638,True,9285165882976373321289124694825334422477391015...,1.0,6983060241552922298703023478823448877784249474...,0.0,2024-08-31 14:00:00,...,4282247823224208661728420876385017155888061913...,0.0,5239744205912208951419911709612814651181011670...,1.0,502.707692,True,3102412140979869086108516105922538862811635984...,0.0,1071771006010210786837770353476734639789497155...,1.0
7,-5976522365870473529,Brighton & Hove Albion,Manchester United,16100.67302,True,4363887430695681438439969777135385836141535812...,1.0,3220601571635084165345439339081203289434711305...,0.0,2024-08-24 11:30:00,...,9571848821720621022067275526860373518879299414...,0.0,6022909429418516783618078939454427585858743426...,1.0,15849.786412,True,9251162583240927984496059688535910213923573518...,0.0,8310108723809550530413844159414015681331768326...,1.0
8,-4534814988459190542,Liverpool,Ipswich Town,49991.555894,True,4076493248006068459372043931314483211652301464...,1.0,1649837615641636063605105083833377683124643888...,0.0,2024-08-17 11:30:00

In [15]:
pre_game_timedelta = pd.Timedelta(minutes=5)


def get_price_at_time(info_df, history_df, token_col, pre_game_timedelta):
    joined_gametime_prices = (
        history_df.reset_index(names=["token_id", "timestamp"])
        .merge(
            info_df[
                [
                    "game_start_time",
                    token_col,
                ]
            ],
            left_on="token_id",
            right_on=token_col,
        )
        .assign(
            timestamp=lambda x: pd.to_datetime(x.timestamp),
        )
        .assign(pre_game_time=lambda x: x.game_start_time - pre_game_timedelta)
    )

    price_at_time = (
        joined_gametime_prices.loc[
            (joined_gametime_prices.timestamp < joined_gametime_prices.pre_game_time)
        ]
        .groupby("token_id")
        .last()
    )
    return price_at_time.price


token_cols_list = [
    "first_win_yes_token_id",
    "first_win_no_token_id",
    "second_win_yes_token_id",
    "second_win_no_token_id",
    "draw_yes_token_id",
    "draw_no_token_id",
]

df = merged_duped.assign(
    **{
        s[:-3] + "_pre_price": get_price_at_time(
            merged_duped, all_histories, s, pre_game_timedelta
        ).values
        for s in token_cols_list
    }
)
df

,match_id,first,second,volume_first_win,closed_first_win,first_win_yes_token_id,first_win_yes_token_price,first_win_no_token_id,first_win_no_token_price,game_start_time,...,draw_yes_token_id,draw_yes_token_price,draw_no_token_id,draw_no_token_price,first_win_yes_token_pre_price,first_win_no_token_pre_price,second_win_yes_token_pre_price,second_win_no_token_pre_price,draw_yes_token_pre_price,draw_no_token_pre_price
0,-8801890004257238758,Fulham,Ipswich Town,5815.749542,True,5390684612591871180226940463614602007632608985...,0.0,4835867034043260323210602260980380172342860099...,1.0,2024-08-31 14:00:00,...,1056239115982502641842514644390919491396338064...,1.0,6474528836620100582363221559257100699919142728...,0.0,0.905,0.500,0.490,0.690,0.275,0.830
1,-8772681732634810279,Bournemouth,Nottingham Forest,1011.850958,True,1050823326941745808232230384776635914686556222...,0.0,3055705040419724043925898401765169215229404101...,1.0,2024-08-17 14:00:00,...,5548849022392522826256320455747757471022017206...,1.0,1287798470161381994661331765600389966380379915...,0.0,0.265,0.825,0.265,0.595,0.170,0.750
2,-8762489910048187249,Arsenal,Brighton & Hove Albion,38583.198239,True,6111503160491403443221654004462481775096015381...,0.0,5891194826219718921556265951457901437359904302...,1.0,2024-08-31 11:30:00,...,3282101053200081500732589041854931815637585813...,1.0,5646917549123102449163419527608818678170441410...,0.0,0.305,0.270,0.370,0.570,0.280,0.735
3,-8627603966281271525,Arsenal,Aston Villa,45331.94036,True,5407858762055642414194294314062033095094505113...,1.0,1926331838542292358683977471374101091601070425...,0.0,2024-08-24 16:30:00,...,3984949472265518240895263056129376916184709390...,0.0,4485337955163659016002637467309784252215764770...,1.0,0.705,0.490,0.310,0.510,0.225,0.750
4,-7758498606970748066,Bournemouth,Everton,4898.660583,True,3959980066742036215089748788937411215762900226...,1.0,2171311100446299993519861709116600368641588070...,0.0,2024-08-31 14:00:00,...,6979466265655015180736494637283075984297819167...,0.0,9201488080747150865847341565897594405713204637...,1.0,0.175,0.435,0.110,0.415,0.265,0.755
5,-7146698704919939652,Brentford,Liverpool,10460.211613,True,8058812937056295301077754963716989842392959855...,0.0,4586745120104545026109118389516543779698202352...,1.0,2024-08-25 15:30:00,...,4937274880027886183807479622344736653709925151...,0.0,8933757800840528696678082589188567084286739295...,1.0,0.510,0.615,0.245,0.425,0.275,0.730
6,-6514747391177184247,Brentford,Southampton,8893.260638,True,9285165882976373321289124694825334422477391015...,1.0,6983060241552922298703023478823448877784249474...,0.0,2024-08-31 14:00:00,...,3102412140979869086108516105922538862811635984...,0.0,1071771006010210786837770353476734639789497155...,1.0,0.200,0.610,0.405,0.195,0.265,0.720
7,-5976522365870473529,Brighton & Hove Albion,Manchester United,16100.67302,True,4363887430695681438439969777135385836141535812...,1.0,3220601571635084165345439339081203289434711305...,0.0,2024-08-24 11:30:00,...,9251162583240927984496059688535910213923573518...,0.0,8310108723809550530413844159414015681331768326...,1.0,0.340,0.690,0.185,0.615,0.295,0.735
8,-4534814988459190542,Liverpool,Ipswich Town,49991.555894,True,4076493248006068459372043931314483211652301464...,1.0,1649837615641636063605105083833377683124643888...,0.0,2024-08-17 11:30:00,...,9540005954158311246396907146026386130907160950...,0.0,2855830619693306362215765671441184555426887207...,1.0,0.715,0.695,0.315,0.665,0.250,0.745
9,-3582618249326489413,Fulham,Manchester United,55641.874117,True,1315385407057815291697000886516025911268371559...,0.0,1406282590488127830418252672774515413854682441...,1.0,2024-08-16 19:00:00,...,4574822642040335406924607620918373394202085737...,0.0,9413900031581212950670543634633325437183025849...,1.0,0.400,0.595,0.245,0.755,0.170,0.770


In [16]:
import numpy as np

true_outcomes = df[
    ["first_win_yes_token_price", "second_win_yes_token_price", "draw_yes_token_price"]
]
predicted_yes_outcomes = df[
    [
        "first_win_yes_token_pre_price",
        "second_win_yes_token_pre_price",
        "draw_yes_token_pre_price",
    ]
]

In [17]:
(predicted_yes_outcomes.round(0).values == true_outcomes.round(0).values).all(
    axis=1
).sum() / len(true_outcomes)

np.float64(0.2)

In [18]:
# df[~(predicted_yes_outcomes.round(0).values == true_outcomes.round(0).values).all(axis=1)]

In [19]:
all_histories.loc[
    "103228697538582971767904061929603928292205175822562237982507803333055995485598"
]

,price
timestamp,
2024-08-21 16:17:01,0.500
2024-08-21 16:18:02,0.900
2024-08-21 16:19:01,0.900
2024-08-21 16:20:02,0.900
2024-08-21 16:21:01,0.900
...,...
2024-08-24 13:55:01,0.905
2024-08-24 13:56:01,0.900
2024-08-24 13:57:01,0.900
